In [1]:
# multi input & output channels
"""
basically, we are carrying on the multi-input channel cross-correlation operations. We do cross-correlation on different
channels with their own kernels separately, and then adds the results together
"""
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # traversing the dimension 0 of the X & K (dimension of channel) and then add them together
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

the operation below can be represented in the following figure
![corr2d_multi_in](../statics/imgs/section6.4_fig1.jpg)

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [3]:
# with the increase of the depth of the model, we always increase the number of the output channels, to decrease the resolution and reach deeper architecture
def corr2d_multi_in_out(X, K):
    # iteration on the 0-dimension of K, and carry on cross-corr in each time. Adding all results together in the end
    return torch.stack([corr2d_multi_in(X, k) for k in K])

In [4]:
K = torch.stack((K, K+1, K+2), 0)  # construct a 3-output-channels kernel by contact kernel tensor K, k+1 and K+2
K.shape

torch.Size([3, 2, 2, 2])

In [5]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [6]:
# 1 * 1 layer could be regarded as FC layer
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # matmul in the FC layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [7]:
# when we execute 1x1 conv, it is just like the corr2d_multi_in_out implemented above
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [9]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
print(f'{Y1} \n {Y2}')
print(f'Y1 shape: {Y1.shape}, Y2 shape: {Y2.shape}')

tensor([[[-1.8546,  0.4771,  1.0454],
         [ 1.2820, -0.7210,  0.9184],
         [ 2.4766,  1.5227, -3.5266]],

        [[ 1.0486, -0.0775, -0.3377],
         [-0.7904,  0.5277, -0.0827],
         [-1.1257, -1.1055,  2.0907]]]) 
 tensor([[[-1.8546,  0.4771,  1.0454],
         [ 1.2820, -0.7210,  0.9184],
         [ 2.4766,  1.5227, -3.5266]],

        [[ 1.0486, -0.0775, -0.3377],
         [-0.7904,  0.5277, -0.0827],
         [-1.1257, -1.1055,  2.0907]]])
Y1 shape: torch.Size([2, 3, 3]), Y2 shape: torch.Size([2, 3, 3])
